In [ ]:
import import_ipynb
import numpy as np
from scipy.sparse import lil_matrix
from fem_processing import master_domain, gaussian_quadrature

# `jacobian()`

In [ ]:
def jacobian(mesh_data, cell):

    # Get the global coordinates of the nodes
    x = [mesh_data['nodes'][node]['xg'][0] for node in cell['conn_sorted']]
    y = [mesh_data['nodes'][node]['xg'][1] for node in cell['conn_sorted']]

    # Calcular o Jacobiano
    Je = np.array([
        [x[1] - x[0], x[2] - x[0]], 
        [y[1] - y[0], y[2] - y[0]]
    ])

    return Je

In [ ]:
def jacobian_tetrahedra(mesh_data, cell):
    """
    Calcula a matriz Jacobiana de um elemento tetraédrico no FEM.

    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - cell: Dicionário com as informações do elemento tetraédrico.

    Retorna:
    - Je: Matriz Jacobiana (3x3).
    """
    
    # Obter as coordenadas globais dos nós do tetraedro
    x = [mesh_data['nodes'][node]['xg'][0] for node in cell['conn_sorted']]
    y = [mesh_data['nodes'][node]['xg'][1] for node in cell['conn_sorted']]
    z = [mesh_data['nodes'][node]['xg'][2] for node in cell['conn_sorted']]

    # Construir a matriz Jacobiana (3x3)
    Je = np.array([
        [x[1] - x[0], x[2] - x[0], x[3] - x[0]], 
        [y[1] - y[0], y[2] - y[0], y[3] - y[0]], 
        [z[1] - z[0], z[2] - z[0], z[3] - z[0]]
    ])

    return Je

# `local_simplified_matrices()`

In [ ]:
def local_simplified_matrices(FINITE_ELEMENT, mesh_data, cell):
    # Initialize the local matrices
    Ne = len(cell['conn_edge'])
    Se = np.zeros((Ne, Ne), dtype='complex128') # Matriz de rigidez do elemento
    Me = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    be = np.zeros((Ne, 1))                      # Vetor de carga do elemento

    # Get the Gauss points and weights
    gauss_points, gauss_weights = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

    # Cálculo da matriz Jacobiana
    # Je = jacobian(mesh_data, cell)
    Je = cell['geo']['jacobian']

    # Determinante e inversa transposta do Jacobiano
    Jdet, JinvT = np.abs(np.linalg.det(Je)), np.linalg.inv(Je).T       

    # Material properties
    pe = cell['p(x)']
    qe = cell['q(x)']
    fe = cell['f(x)']

    # Matriz de rigidez do elemento
    Se = 2 / Jdet * np.array([
        [1, -1, 1], 
        [-1, 1, -1],
        [1, -1, 1]
    ])

    # Integração numérica
    for xik, wk in zip(gauss_points, gauss_weights):
        # Função de forma vetorial no ponto de integração
        phi_hat = master_domain.shape_functions_n0(*xik)

        # Montagem da matriz de massa
        for i in range(Ne):
            # Transformação de Piola Covariante
            phi_i = JinvT @ phi_hat[i]

            # Vetor de cargas
            be[i] += (fe.T @ phi_i).item() * Jdet * wk
            
            for j in range(Ne):
                # Transformação de Piola Covariante
                phi_j = JinvT @ phi_hat[j]

                # Matriz de massa
                Me[i, j] += (phi_i.T @ phi_j).item() * Jdet * wk 
    
    return Se, be, Me

# `local_matrices()`

In [ ]:
def local_matrices(FINITE_ELEMENT, cell):
    # Initialize the local matrices
    Ne = len(cell['conn_edge'])
    Se = np.zeros((Ne, Ne), dtype='complex128')     # Matriz de rigidez do elemento
    Me = np.zeros((Ne, Ne), dtype='complex128')     # Matriz de massa do elemento
    be = np.zeros((Ne, 1), dtype='complex128')      # Vetor de carga do elemento

    # Define the Finite Element Type
    ElementType, ElementOrder = FINITE_ELEMENT

    # Material properties
    pe = cell['p(x)']
    qe = cell['q(x)']
    fe = cell['f(x)']

    # Cálculo da matriz Jacobiana Completa 3x3
    # Je = assembly.jacobian(mesh_data, cell)
    Je = cell['geo']['jacobian']

    # Determinante e inversa transposta do Jacobiano
    Jdet, JinvT = np.abs(np.linalg.det(Je)), np.linalg.inv(Je).T 

    # Rotacional das funções de forma
    curl_phi_hat = [
        np.array([[0], [0], [2]]),
        np.array([[0], [0], [-2]]),
        np.array([[0], [0], [2]])
    ]
    
    # Tensor Métrico Covariante
    # Produto interno dos vetores coluna de Je: t1 dot t2
    Gt = Je.T @ Je

    # Tensor métrico contravariante
    Gn = JinvT.T @ JinvT

    # Integração numérica
    gauss_points, gauss_weights = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
    for xik, wk in zip(gauss_points, gauss_weights):
        # Função de forma vetorial no ponto de integração
        if ElementType == 'Triangle':
            phi_hat = master_domain.shape_functions_nedelec_3d(*xik)
        elif ElementType == 'Tetrahedron':
            phi_hat = master_domain.shape_functions_nedelec_tetrahedra(*xik)

        for i in range(Ne):
            # Transformação de Piola Covariante
            phi_i = qe @ JinvT @ phi_hat[i]
            curl_phi_i = pe * 1/Jdet * Je @ curl_phi_hat[i]

            # Vetor de cargas
            be[i] += (fe.T @ phi_i).item() * Jdet * wk
            
            for j in range(Ne):
                # Transformação de Piola Covariante
                phi_j = JinvT @ phi_hat[j]
                curl_phi_j = 1/Jdet * Je @ curl_phi_hat[j]

                # Matriz de rigidez
                #Se[i, j] =+ (curl_phi_hat[i].T @ Gt @ curl_phi_hat[j]).item() * (1/Jdet) * wk
                Se[i, j] += (curl_phi_i.T @ curl_phi_j).item() * Jdet * wk

                # Matriz de massa
                # Me[i, j] += (phi_i.T @ phi_j).item() * Jdet * wk
                Me[i, j] += (phi_hat[i].T @ Gn @ phi_hat[j]).item() * Jdet * wk

    # Matriz de rigidez do elemento
    Se1 = (1/Jdet) * 0.5 * np.array([
        [4, -4, 4], 
        [-4, 4, -4],
        [4, -4, 4]
    ])
    
    return Se, Me, be

# `reduced_global_matrices()`

In [ ]:
def reduced_global_matrices(FINITE_ELEMENT, mesh_data):
    # Arestas livres
    free_edges = {key: value
                   for key, value in mesh_data['edges'].items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_edge: idx
                          for idx, global_edge in enumerate(free_edges.keys())}

    # Initialize the global matrix and vector
    Sgr = lil_matrix((len(free_edges), len(free_edges)), dtype='complex128')
    Mgr = lil_matrix((len(free_edges), len(free_edges)), dtype='complex128')
    bgr = lil_matrix((len(free_edges), 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values():
        # Compute the local data
        Se, Me, be = local_matrices(FINITE_ELEMENT, mesh_data, cell)

        for i, ig in enumerate(cell['conn_edge']):
            if ig in free_edges:
                ig_red = global_to_reduced[ig]
                
                for j, jg in enumerate(cell['conn_edge']):
                    if jg in free_edges:
                        jg_red = global_to_reduced[jg]
                        Sgr[ig_red, jg_red] += Se[i, j]
                        Mgr[ig_red, jg_red] += Me[i, j]

                    # A aresta global é de Dirichlet
                    else:
                        # Contribuição de Dirichlet para o vetor reduzido
                        bgr[ig_red, 0] += -(Se[i, j] + Me[i, j]) * mesh_data['edges'][jg]['bc']['value']
                
                # preenche o vetor global de carga
                bgr[ig_red, 0] += be[i]

    return Sgr, Mgr, bgr

# `global_potentials()`

In [ ]:
def global_potentials(mesh_data, curl_uhr):
    # Inicializa o vetor de potenciais globais com zeros
    curl_uh = {}

    # Arestas livres
    free_edges = {key: value
                   for key, value in mesh_data['edges'].items() if value['bc']['type'] != 'Dirichlet'}

    # Mapeamento de índices globais para índices reduzidos
    global_to_reduced = {global_edge: idx
                         for idx, global_edge in enumerate(free_edges.keys())}

    # Preenche o vetor solução
    for key, edge in mesh_data['edges'].items():
        # Verifica se a aresta é livre
        if key in free_edges:
            # Mapeamento global para reduzido
            curl_uh[int(key)] = curl_uhr[global_to_reduced[key]]

        # Nó de Dirichlet
        else:
            curl_uh[int(key)] = edge['bc']['value']

    return curl_uh

# `apply_simple_dirichlet()`

In [ ]:
def apply_simple_dirichlet(Sg, Mg, bg, mesh_data):
    for key, edge in mesh_data['edges'].items():
        # Verifica se a aresta é de Dirichlet
        if edge['bc']['type'] == 'Dirichlet':
            e = int(key) - 1
            
            # Zera a linha e coluna correspondente a aresta de Dirichlet
            Sg[e, :] = 0
            Mg[e, :] = 0
            
            # Define o valor 1 na diagonal para evitar singularidade
            Sg[e, e] = 0.5
            Mg[e, e] = 0.5
            
            # Ajusta o vetor de forças com o valor de Dirichlet
            bg[e] = edge['bc']['value']

    return Sg, Mg, bg

# `global_matrices()`

In [ ]:
def global_matrices(FINITE_ELEMENT, mesh_data):
    # Inicializa a matriz global como uma matriz esparsa zero (tamanho NxN)
    Nedges = len(mesh_data['edges'])
    Sg = lil_matrix((Nedges, Nedges), dtype='complex128')
    Mg = lil_matrix((Nedges, Nedges), dtype='complex128')
    bg = lil_matrix((Nedges, 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values(): 
        # Computa as matrizes locais
        Se, Me, be = local_matrices(FINITE_ELEMENT, mesh_data, cell)

        # loop sobre os nós locais de cada elemento
        for i, ig in enumerate(cell['conn_edge']):
            ig = int(ig)-1
            for j, jg in enumerate(cell['conn_edge']):
                jg = int(jg)-1
                Sg[ig, jg] += Se[i, j]
                Mg[ig, jg] += Me[i, j]
                
            # preenche o vetor global b
            bg[ig, 0] += be[i]

    return Sg, Mg, bg

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil, 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.